In [1]:
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

import pandas as pd
import numpy as np

import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

select agents for comparison

In [3]:
agents = dict(
    eight='Models/Victim/default_PPO_citylearn_challenge_2022_phase_2_Building_6_8_bins_130.zip',
    ten='Models/Victim/default_PPO_citylearn_challenge_2022_phase_2_Building_6_15_bins_200.zip',
    fifteen='Models/Victim/default_PPO_citylearn_challenge_2022_phase_2_Building_6_15_bins_200.zip',
    twenty='20 bin PPO 300 results/default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_0.zip',
    twenty_four='Models/Victim/default_PPO_citylearn_challenge_2022_phase_2_Building_6_24_bins_300.zip',
)
for name, path in agents.items():
    agents[name] = PPO.load(path=path)

Each agent needs a separate env for parallelization, otherwise they all try to use the same one

In [9]:
schema = DataSet.get_schema(DATASET_NAME)
envs = dict()
for name, agent in agents.items():
    envs[name] = utils.make_discrete_env(schema=schema,  
                            action_bins=agent.action_space[0].n,
                            seed=42)

In [5]:
%%time
results = Parallel(n_jobs=len(agents), prefer='threads')(delayed(utils.eval_agent)(env,agent) for env, agent in zip(envs.values(), agents.values()))

CPU times: total: 3min 43s
Wall time: 2min 58s


In [6]:
kpis = [results[i][0] for i in range(len(results))]

In [7]:
df_kpis = pd.concat(kpis, axis='columns',keys=agents.keys())

In [8]:
df_kpis.to_csv('bin_comparison_KPIs.csv')